### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[ 10.  10.  10.  10.]
 [ 10.   4.   9.  10.]
 [  8.   9.  10.   5.]
 [  9.   8.  10.  10.]
 [ 10.   5.   9.   9.]
 [  6.   4.  10.   6.]
 [  9.   8.  10.   9.]
 [ 10.   5.   9.   8.]
 [  7.   8.  10.   8.]
 [  9.   5.   9.   7.]
 [  9.   8.  10.   8.]
 [  9.  10.  10.   9.]
 [ 10.   9.  10.   8.]
 [  5.   8.   5.   8.]
 [ 10.   8.  10.  10.]
 [  9.   9.  10.  10.]
 [  9.   8.   8.   8.]
 [ 10.   8.   1.  10.]
 [  5.   6.  10.  10.]
 [  8.   7.  10.   7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the function on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [2]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0] # number of rows in the matrix
    n_movies = ratings_mat.shape[1] # number of movies in the matrix
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat)) # total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users, latent_features)  # user matrix filled with random values of shape user x latent 
    movie_mat = np.random.rand(latent_features, n_movies) # movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimization Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):
        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                
                # if the rating exists
                if ratings_mat[i, j] > 0:                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    # Keep track of the total sum of squared errors for the matrix
                    sse_accum += diff ** 2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2 * diff * movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * ( 2 * diff * user_mat[i, k])

        # print results for iteration
        print("%d \t\t %f" % (iteration + 1, sse_accum/num_ratings))
        
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [5]:
# use your function with 4 latent features, lr of 0.005 and 10 iterations
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=10)

Optimization Statistics
Iterations | Mean Squared Error 
1 		 43.955216
2 		 14.661683
3 		 4.003727
4 		 2.878395
5 		 2.784662
6 		 2.763448
7 		 2.752460
8 		 2.742042
9 		 2.730714
10 		 2.718452


In [6]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 10.15187332   8.98103936  10.58137138  10.0832377 ]
 [  8.65300776   7.63939847   9.06619403   8.68492693]
 [  7.89847648   6.86150772   8.19029165   7.82126454]
 [  9.26857142   8.22852953  10.06914315   9.76345964]
 [  8.3423178    7.39645785   8.97164628   8.68474078]
 [  6.53322441   5.70777407   7.31009704   7.11304672]
 [  9.46218472   8.29602059   9.54057677   9.05275913]
 [  8.28775437   7.30745284   8.56097595   8.14270553]
 [  8.54366962   7.59994587   8.82228092   8.44194798]
 [  7.910255     6.75245769   8.05102545   7.54711308]
 [  8.73227927   7.51097893   9.39469316   9.01724984]
 [  9.82517543   8.57036726   9.94257683   9.44666941]
 [  9.26006392   8.02761181   9.74406169   9.29749188]
 [  6.57596934   5.89063728   6.93071206   6.62876898]
 [  9.96642053   8.70686565  10.10304625   9.62565221]
 [  9.51358641   8.32890854  10.32344786   9.90853739]
 [  8.64349633   7.33001636   8.63578356   8.09527262]
 [  7.00325841   6.17586518   7.42722385   7.1848334 ]
 [  8.2438

**Write your summary here.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [3]:
#use your function with 4 latent features, lr of 0.005 and 250 iterations
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimization Statistics
Iterations | Mean Squared Error 
1 		 40.799776
2 		 13.097975
3 		 3.734060
4 		 2.757070
5 		 2.669933
6 		 2.645146
7 		 2.624758
8 		 2.600824
9 		 2.572242
10 		 2.538870
11 		 2.500513
12 		 2.456873
13 		 2.407630
14 		 2.352527
15 		 2.291437
16 		 2.224413
17 		 2.151736
18 		 2.073936
19 		 1.991793
20 		 1.906312
21 		 1.818662
22 		 1.730093
23 		 1.641837
24 		 1.555016
25 		 1.470567
26 		 1.389202
27 		 1.311402
28 		 1.237451
29 		 1.167486
30 		 1.101554
31 		 1.039660
32 		 0.981804
33 		 0.927990
34 		 0.878230
35 		 0.832523
36 		 0.790846
37 		 0.753130
38 		 0.719256
39 		 0.689047
40 		 0.662276
41 		 0.638673
42 		 0.617941
43 		 0.599767
44 		 0.583839
45 		 0.569853
46 		 0.557524
47 		 0.546588
48 		 0.536810
49 		 0.527977
50 		 0.519904
51 		 0.512426
52 		 0.505401
53 		 0.498703
54 		 0.492221
55 		 0.485857
56 		 0.479522
57 		 0.473137
58 		 0.466628
59 		 0.459929
60 		 0.452975
61 		 0.445710
62 		 0.438079
63 		 0.430031
64 		

In [4]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 10.00002097  10.0000081   10.00000451   9.99998077]
 [ 10.00002236   4.00000903   9.00000497   9.99997886]
 [  8.00003834   9.00001502  10.0000082    4.99996418]
 [  9.0000041    8.00000025   9.9999996    9.99999627]
 [  9.99994648   4.99997708   8.99998663   9.00005   ]
 [  6.00005855   4.00002401  10.00001345   5.99994487]
 [  8.9999787    7.99998998   9.99999388   9.00002007]
 [  9.99999324   4.99999693   8.99999808   8.00000615]
 [  6.99999583   7.99999736   9.9999982    8.00000406]
 [  8.9999968    4.99999844   8.999999     7.00000288]
 [  8.9999549    7.99998081   9.999989     8.00004246]
 [  8.99993567   9.99997344   9.99998513   9.00006076]
 [  9.99995749   8.99998374   9.99999147   8.00004027]
 [  4.99997897   7.99999223   4.99999624   8.0000205 ]
 [ 10.00000084   8.00000242  10.00000239   9.99999969]
 [  9.00002572   9.00001236  10.00000789   9.99997645]
 [  9.00005872   8.00002527   8.00001486   7.99994539]
 [ 10.00001968   8.00000826   1.00000499   9.99998238]
 [  5.0000

**Write your summary here.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [8]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[ nan,  10.,  10.,  10.],
        [ 10.,   4.,   9.,  10.],
        [  8.,   9.,  10.,   5.],
        [  9.,   8.,  10.,  10.],
        [ 10.,   5.,   9.,   9.],
        [  6.,   4.,  10.,   6.],
        [  9.,   8.,  10.,   9.],
        [ 10.,   5.,   9.,   8.],
        [  7.,   8.,  10.,   8.],
        [  9.,   5.,   9.,   7.],
        [  9.,   8.,  10.,   8.],
        [  9.,  10.,  10.,   9.],
        [ 10.,   9.,  10.,   8.],
        [  5.,   8.,   5.,   8.],
        [ 10.,   8.,  10.,  10.],
        [  9.,   9.,  10.,  10.],
        [  9.,   8.,   8.,   8.],
        [ 10.,   8.,   1.,  10.],
        [  5.,   6.,  10.,  10.],
        [  8.,   7.,  10.,   7.]])

In [9]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimization Statistics
Iterations | Mean Squared Error 
1 		 40.124861
2 		 13.178226
3 		 3.967496
4 		 2.910781
5 		 2.785120
6 		 2.741391
7 		 2.711549
8 		 2.683405
9 		 2.653392
10 		 2.619983
11 		 2.582180
12 		 2.539136
13 		 2.490064
14 		 2.434239
15 		 2.371032
16 		 2.299961
17 		 2.220765
18 		 2.133471
19 		 2.038476
20 		 1.936595
21 		 1.829091
22 		 1.717649
23 		 1.604300
24 		 1.491282
25 		 1.380870
26 		 1.275181
27 		 1.176006
28 		 1.084688
29 		 1.002069
30 		 0.928495
31 		 0.863886
32 		 0.807824
33 		 0.759663
34 		 0.718623
35 		 0.683868
36 		 0.654564
37 		 0.629920
38 		 0.609212
39 		 0.591790
40 		 0.577088
41 		 0.564615
42 		 0.553954
43 		 0.544751
44 		 0.536712
45 		 0.529589
46 		 0.523175
47 		 0.517298
48 		 0.511811
49 		 0.506591
50 		 0.501534
51 		 0.496547
52 		 0.491551
53 		 0.486474
54 		 0.481252
55 		 0.475824
56 		 0.470136
57 		 0.464137
58 		 0.457779
59 		 0.451016
60 		 0.443805
61 		 0.436109
62 		 0.427891
63 		 0.419121
64 		

In [10]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 8.783137656621198:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [11]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=4, learning_rate=0.005, iters=20)

Optimization Statistics
Iterations | Mean Squared Error 
1 		 23.357607
2 		 10.769414
3 		 7.362719
4 		 5.670296
5 		 4.608753
6 		 3.866714
7 		 3.313570
8 		 2.883966
9 		 2.540914
10 		 2.261548
11 		 2.030790
12 		 1.838168
13 		 1.676063
14 		 1.538703
15 		 1.421588
16 		 1.321140
17 		 1.234480
18 		 1.159274
19 		 1.093623
20 		 1.035973


`6.` Now that you have a set of predictions for each user-movie pair,  let's answer a few questions about your results. Provide the correct values for each of the variables below, and check your solutions using the tests below.

In [12]:
# How many actual ratings exist in first_1000_users
num_ratings = np.count_nonzero(~np.isnan(first_1000_users))
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()

# How many ratings did we make for user-movie pairs that didn't actually have ratings
ratings_for_missing = first_1000_users.shape[0] * first_1000_users.shape[1] - num_ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [13]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
